In [1]:
import pandas as pd
import numpy as np

In [2]:
%pwd

'/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/src/analysis/round4'

In [4]:
%cd ../../../data/data-bottles/round4/

/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/data/data-bottles/round4


In [5]:
%ls

__init__.py               prices_round_4_day_3.csv  trades_round_4_day_3.csv
prices_round_4_day_1.csv  trades_round_4_day_1.csv
prices_round_4_day_2.csv  trades_round_4_day_2.csv


In [6]:
df = pd.read_csv('prices_round_4_day_3.csv', delimiter = ';')

In [ ]:
df = df[df['product'] == 'MAGNIFICENT_MACARONS']

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,3,0,SQUID_INK,1820.0,1.0,1819.0,27.0,NaN,NaN,1822,27,NaN,NaN,NaN,NaN,1821.0,0.0
1,3,0,VOLCANIC_ROCK_VOUCHER_10000,182.0,16.0,NaN,NaN,NaN,NaN,183,16,NaN,NaN,NaN,NaN,182.5,0.0
2,3,0,KELP,2040.0,28.0,NaN,NaN,NaN,NaN,2043,27,NaN,NaN,NaN,NaN,2041.5,0.0
3,3,0,RAINFOREST_RESIN,9992.0,33.0,NaN,NaN,NaN,NaN,10008,33,NaN,NaN,NaN,NaN,10000.0,0.0
4,3,0,VOLCANIC_ROCK_VOUCHER_10250,36.0,16.0,NaN,NaN,NaN,NaN,37,16,NaN,NaN,NaN,NaN,36.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,3,999900,JAMS,6446.0,96.0,6445.0,175.0,NaN,NaN,6447,261,NaN,NaN,NaN,NaN,6446.5,0.0
149996,3,999900,CROISSANTS,4295.0,148.0,NaN,NaN,NaN,NaN,4296,62,4297.0,86.0,NaN,NaN,4295.5,0.0
149997,3,999900,VOLCANIC_ROCK_VOUCHER_9500,421.0,20.0,NaN,NaN,NaN,NaN,422,20,NaN,NaN,NaN,NaN,421.5,0.0
149998,3,999900,VOLCANIC_ROCK,9920.0,137.0,9919.0,69.0,NaN,NaN,9921,131,9922.0,69.0,NaN,NaN,9920.5,0.0
